![](https://miro.medium.com/max/2400/1*DOv2T74U6C3fd1EoEN7LoA.gif)

# <p style="text-align: center;"> Table of Contents </p>

- ## 1. [Introduction](#Introduction)
   - ### 1.1 [Abstract](#abstract)
   - ### 1.2 [Importing Libraries](#import)

- ### 2 [Q-Learning](#q)
   - ### 2.1 [Why Deep Q-Learning?](#wq)
   - ### 2.2 [Deep Q-Network](#deepq)
   - ### 2.3 [Challenges in Deep RL as Compared to Deep Learning](#cc)
       - #### 2.3.1 [Target Network](#tr)
       - #### 2.3.2 [Experience Replay](#er)
       - #### 2.3.3 [Putting it together](#pt)
       
- ## 3. [Conclusion](#Conclusion)
- ## 4. [Contribution](#Contribution)
- ## 5. [Citation](#Citation)
- ## 6. [License](#License)

In [ ]:
from IPython.core.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## <a id='Introduction'> 1. Introduction </a>

## <a id="abstract"> 1.1 Abstract </a>

This repository trains a q deep learning network from the game 2048 and plots a performance graph. The gamelogic of the game 2048 is based on the implementation from Georg Wiese on his GitHub Repo . The deep q learning code is loosely based on the implementation form this GitHub Repo tutorial and was greatly enhanced and adapted to include the game 2048.


## <a id="import"> 1.2 Importing Libaries </a>

<br>First, we import the libraries and the gamelogic class. As mentioned before, the whole code can be run in the section "Train it" at the end of this description. The dependencies are also listed in requirements.txt for an easy install with pip.

In [11]:

import itertools
import logging
import numpy as np
from collections import deque
import itertools
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import json
import time
from shutil import copyfile
import parameters
import os
#from gamelogic.game Game


## <a id="q"> 2.1 Q-Learning </a>

Let’s say we know the expected reward of each action at every step. This would essentially be like a cheat sheet for the agent! Our agent will know exactly which action to perform.

It will perform the sequence of actions that will eventually generate the maximum total reward. This total reward is also called the Q-value and we will formalise our strategy as:

![](images/DQN_1.png)


The above equation states that the Q-value yielded from being at state s and performing action a is the immediate reward r(s,a) plus the highest Q-value possible from the next state s’. Gamma here is the discount factor which controls the contribution of rewards further in the future.

Q(s’,a) again depends on Q(s”,a) which will then have a coefficient of gamma squared. So, the Q-value depends on Q-values of future states as shown here:

![](images/DQN_2.png)

Adjusting the value of gamma will diminish or increase the contribution of future rewards.

Since this is a recursive equation, we can start with making arbitrary assumptions for all q-values. With experience, it will converge to the optimal policy. In practical situations, this is implemented as an update:

![](images/DQN_3.png)

where alpha is the learning rate or step size. This simply determines to what extent newly acquired information overrides old information.



#### Reward
We tried both the score and the maximum value of the tile as the reward value. The score is calculated in a way that it increases every time by the value of the newly merged tiles. If for example two 4s are merged the reward is 8.
This means the score partly represents the highest value on the board but gives also an incentive to have multiple high value tiles compared to just looking at the maximum value present at the board.

For our algorithm, we implemented both versions of the reward, which results in a optimization of the score or a win-lose classification with a fixed target respectively.

As the results below show, we achieved similar results with both approaches.

#### Loss
In order to logically represent this intuition and train it, we need to express this as a formula that we can optimize on. The loss is just a value that indicates how far our prediction is from the actual target. For example, the prediction of the model could indicate that it sees more value in swiping left when in fact it can gain more reward by swiping upwards. We want to decrease this gap between the prediction and the target (loss). The loss is defined as:

$$ L = \frac{1}{2} \underbrace{	[r + \gamma*  max_{a'}Q(s',a')}_{\mathrm{target}} -\underbrace{Q(s,a)}_{\mathrm{prediction}}]^2 $$

whith the target q value looking like this in the code: <br>
target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))

## <a id="wq"> 2.2 Why ‘Deep’ Q-Learning? </a>

Q-learning is a simple yet quite powerful algorithm to create a cheat sheet for our agent. This helps the agent figure out exactly which action to perform.

But what if this cheatsheet is too long? Imagine an environment with 10,000 states and 1,000 actions per state. This would create a table of 10 million cells. Things will quickly get out of control!

It is pretty clear that we can’t infer the Q-value of new states from already explored states. This presents two problems:

First, the amount of memory required to save and update that table would increase as the number of states increases
Second, the amount of time required to explore each state to create the required Q-table would be unrealistic

## <a id="deepq">2.3 Deep Q-Networks </a>

In deep Q-learning, we use a neural network to approximate the Q-value function. The state is given as the input and the Q-value of all possible actions is generated as the output. The comparison between Q-learning & deep Q-learning is wonderfully illustrated below:

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/04/Screenshot-2019-04-16-at-5.46.01-PM-850x558.png)

So, what are the steps involved in reinforcement learning using deep Q-learning networks (DQNs)?

#### - 1.All the past experience is stored by the user in memory
#### - 2. The next action is determined by the maximum output of the Q-network
#### - 3. The loss function here is mean squared error of the predicted Q-value and the target Q-value – Q*. This is basically a regression problem. However, we do not know the target or actual value here as we are dealing with a reinforcement learning problem. Going back to the Q-value update equation derived fromthe Bellman equation. we have:

![](images/DQN_4.png)

`The section in green represents the target. We can argue that it is predicting its own value, but since R is the unbiased true reward, the network is going to update its gradient using backpropagation to finally converge.`



## <a id="cc"> 2.4 Challenges in Deep RL as Compared to Deep Learning </a>

- #### Non-stationary or unstable target: Let us go back to the pseudocode for deep Q-learning:
![](images/DQN_5.png)
As you can see in the above code, the target is continuously changing with each iteration. In deep learning, the target variable does not change and hence the training is stable, which is just not true for RL.

To summarise, we often depend on the policy or value functions in reinforcement learning to sample actions. However, this is frequently changing as we continuously learn what to explore. As we play out the game, we get to know more about the ground truth values of states and actions and hence, the output is also changing.

### <a id="tr"> 2.4.1 Target Network</a> 
Since the same network is calculating the predicted value and the target value, there could be a lot of divergence between these two. So, instead of using 1one neural network for learning, we can use two.

We could use a separate network to estimate the target. This target network has the same architecture as the function approximator but with frozen parameters. For every C iterations (a hyperparameter), the parameters from the prediction network are copied to the target network. This leads to more stable training because it keeps the target function fixed (for a while):

![](images/DQN_6.png)

### <a id="er">2.4.2 Experience Replay</a>

![](images/DQN_7.png)
What does the above statement mean? Instead of running Q-learning on state/action pairs as they occur during simulation or the actual experience, the system stores the data discovered for [state, action, reward, next_state] – in a large table.

Let’s understand this using an example.

Suppose we are trying to build a video game bot where each frame of the game represents a different state. During training, we could sample a random batch of 64 frames from the last 100,000 frames to train our network. This would get us a subset within which the correlation amongst the samples is low and will also provide better sampling efficiency.

### <a id="pt"> 2.4.3 Putting it all Together </a>

The concepts we have learned so far? They all combine to make the deep Q-learning algorithm that was used to achive human-level level performance in Atari games (using just the video frames of the game).

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/04/Screenshot-2019-04-17-at-1.15.28-PM.png)

#### we have listed the steps involved in a deep Q-network (DQN) below:

> - 1. Preprocess and feed the game screen (state s) to our DQN, which will return the Q-values of all possible actions in the state
> - 2. Select an action using the epsilon-greedy policy. With the probability epsilon, we select a random action a and with probability 1-epsilon, we select an action that has a maximum Q-value, such as a = argmax(Q(s,a,w))
>- 3. Perform this action in a state s and move to a new state s’ to receive a reward. This state s’ is the preprocessed image of the next game screen. We store this transition in our replay buffer as <s,a,r,s’>
>- 4. Next, sample some random batches of transitions from the replay buffer and calculate the loss
>- 5.It is known that: which is just the squared difference between target Q and predicted Q
>- 6. Perform gradient descent with respect to our actual network parameters in order to minimize this loss
>- 7. After every C iterations, copy our actual network weights to the target network weights
>- 8. Repeat these steps for M number of episodes

##  <a id="Citation"> 5. Citations </a>

- https://github.com/SergioIommi/DQN-2048
- https://github.com/berkay-dincer/2048-ai-mcts/tree/master/src
- https://github.com/Anaconda-Platform/nb_conda_kernels
- https://towardsdatascience.com/welcome-to-deep-reinforcement-learning-part-1-dqn-c3cab4d4
- http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html
    

##  <a id="License"> 6. License </a>
Copyright (c) 2020 Manali Sharma, Rushabh Nisher

 

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

 

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

 

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.